In [498]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database connection

In [499]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [500]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [501]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction

In [502]:
cliente = pd.read_sql_table('dim_cliente', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
tiempo = pd.read_sql_table('dim_tiempo', etl_conn)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [503]:
total_nan = servicio.isna().sum()
print(total_nan)

id                                    0
descripcion                           0
nombre_solicitante                    0
fecha_solicitud                       0
hora_solicitud                        0
fecha_deseada                         6
hora_deseada                          0
nombre_recibe                         0
telefono_recibe                       0
descripcion_pago                    260
ida_y_regreso                         0
activo                                0
novedades                           336
cliente_id                            0
destino_id                            0
mensajero_id                        727
origen_id                             0
tipo_pago_id                          0
tipo_servicio_id                      0
tipo_vehiculo_id                      0
usuario_id                            0
prioridad                             0
ciudad_destino_id                     0
ciudad_origen_id                      0
hora_visto_por_mensajero          28428


## Transformation

In [504]:
tiempo_unique = tiempo[["hour", "key_tiempo"]].drop_duplicates(subset=["hour"])


# Convertir 'hora_solicitud' a string en el formato HH:MM:SS
# Convertir 'hora_solicitud' a datetime
servicio["hora_solicitud"] = pd.to_datetime(servicio["hora_solicitud"], format='%H:%M:%S')


# Extraer la hora como un entero (1-24)
servicio["hora"] = servicio["hora_solicitud"].dt.hour

servicio = servicio[["id", "hora", "cliente_id", "mensajero_id"]]
servicio.fillna(-1, inplace=True)

# Hacer el merge asegurando que coincidan
hecho_servicio_hora = pd.merge(servicio, tiempo_unique[["hour", "key_tiempo"]], left_on="hora", right_on="hour", how="left")
hecho_servicio_hora.head(5)

,id,hora,cliente_id,mensajero_id,hour,key_tiempo
0,34,9,5,-1.0,9,9
1,35,11,5,7.0,11,11
2,36,19,5,-1.0,19,19
3,41,9,5,-1.0,9,9
4,42,9,5,-1.0,9,9


In [505]:
hecho_servicio_hora = pd.merge(hecho_servicio_hora, cliente[["cliente_id", "key_cliente"]], left_on="cliente_id", right_on="cliente_id", how="left")
hecho_servicio_hora.head(5)

,id,hora,cliente_id,mensajero_id,hour,key_tiempo,key_cliente
0,34,9,5,-1.0,9,9,7
1,35,11,5,7.0,11,11,7
2,36,19,5,-1.0,19,19,7
3,41,9,5,-1.0,9,9,7
4,42,9,5,-1.0,9,9,7


In [506]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
hecho_servicio_hora = hecho_servicio_hora.merge(mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
hecho_servicio_hora.fillna(-1, inplace=True)
hecho_servicio_hora.head(5)

,id,hora,cliente_id,mensajero_id,hour,key_tiempo,key_cliente,key_mensajero
0,34,9,5,-1.0,9,9,7,-1.0
1,35,11,5,7.0,11,11,7,13.0
2,36,19,5,-1.0,19,19,7,-1.0
3,41,9,5,-1.0,9,9,7,-1.0
4,42,9,5,-1.0,9,9,7,-1.0


In [507]:
print(len(hecho_servicio_hora))

28430


## Solicitudes_por_hora

In [508]:
# Contar solicitudes por cada 'año_mes'
solicitudes_por_hora = hecho_servicio_hora.groupby("hora").size().reset_index(name="solicitud_por_hora")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original

hecho_servicio_hora = hecho_servicio_hora.merge(solicitudes_por_hora, on="hora", how="left")

hecho_servicio_hora.head(5)




,id,hora,cliente_id,mensajero_id,hour,key_tiempo,key_cliente,key_mensajero,solicitud_por_hora
0,34,9,5,-1.0,9,9,7,-1.0,3391
1,35,11,5,7.0,11,11,7,13.0,3295
2,36,19,5,-1.0,19,19,7,-1.0,346
3,41,9,5,-1.0,9,9,7,-1.0,3391
4,42,9,5,-1.0,9,9,7,-1.0,3391


In [509]:
print(len(hecho_servicio_hora))

28430


## Solicitud_por_hora_cliente

In [510]:
# Contar solicitudes por cada 'key_tiempo' y 'key_cliente' 
solicitud_por_hora_cliente = hecho_servicio_hora.groupby(["key_tiempo","key_cliente"]).size().reset_index(name="solicitud_por_hora_cliente")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_hora = hecho_servicio_hora.merge(solicitud_por_hora_cliente, on=["key_cliente", "key_tiempo"], how="left")
hecho_servicio_hora.head(5)

,id,hora,cliente_id,mensajero_id,hour,key_tiempo,key_cliente,key_mensajero,solicitud_por_hora,solicitud_por_hora_cliente
0,34,9,5,-1.0,9,9,7,-1.0,3391,660
1,35,11,5,7.0,11,11,7,13.0,3295,862
2,36,19,5,-1.0,19,19,7,-1.0,346,25
3,41,9,5,-1.0,9,9,7,-1.0,3391,660
4,42,9,5,-1.0,9,9,7,-1.0,3391,660


## Solicitud_por_mensajero

In [511]:
# Contar solicitudes por cada 'key_tiempo' y 'key_mensajero'
cantidad_solicitud_por_mensajero = hecho_servicio_hora.groupby(["key_mensajero"]).size().reset_index(name="cantidad_solicitud_por_mensajero")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_hora.merge(cantidad_solicitud_por_mensajero, on=["key_mensajero"], how="left")
hecho_servicio_mensual.head(6)

,id,hora,cliente_id,mensajero_id,hour,key_tiempo,key_cliente,key_mensajero,solicitud_por_hora,solicitud_por_hora_cliente,cantidad_solicitud_por_mensajero
0,34,9,5,-1.0,9,9,7,-1.0,3391,660,727
1,35,11,5,7.0,11,11,7,13.0,3295,862,68
2,36,19,5,-1.0,19,19,7,-1.0,346,25,727
3,41,9,5,-1.0,9,9,7,-1.0,3391,660,727
4,42,9,5,-1.0,9,9,7,-1.0,3391,660,727
5,43,10,5,-1.0,10,10,7,-1.0,2932,360,727


## Eliminar filas

In [512]:
hecho_servicio_hora.drop(columns=["id","cliente_id","mensajero_id","hour"], inplace=True)
hecho_servicio_hora

,hora,key_tiempo,key_cliente,key_mensajero,solicitud_por_hora,solicitud_por_hora_cliente
0,9,9,7,-1.0,3391,660
1,11,11,7,13.0,3295,862
2,19,19,7,-1.0,346,25
3,9,9,7,-1.0,3391,660
4,9,9,7,-1.0,3391,660
...,...,...,...,...,...,...
28425,10,10,11,8.0,2932,2214
28426,16,16,11,27.0,2124,1683
28427,7,7,7,32.0,1066,293
28428,10,10,11,18.0,2932,2214


## Load

In [513]:
hecho_servicio_hora.to_sql("hecho_servicio_hora", etl_conn, if_exists="replace", index_label="key_solicitud_servicio") 

430